# Data Collection - Data Sources Exploration

This is a rough notebook to get started in identifying, downloading data sources.
Also early version of functions needed will be coded in there before being moved to their relevant files.

The market data on the current shopping list is:

Sovereigns
-------------------
- US treasury note yields [wip]
- UK gilts [wip]
- DE bonds
- FR bonds

Inflation
-------------------
- EU CPI [wip]
- UK CPI [wip]
- US CPI  [wip]

Interest Rates
-------------------
- ECB [wip]
- BOE [wip]

FX Rates
-------------------
- EUR/GBP [wip]
- USD/EUR [wip]

Equity Indices
-------------------
- FTSE100 [wip]
- NASDAQ [wip]
- DOW [wip]
- S&P500 [wip]

Volatility
-------------------

Commodities 
-------------------
- Brent [wip]
- Grain [wip]
- Aluminium [wip]

GDP
-------------------

Production
-------------------

Confidence
-------------------


Known data sources
-------------------
Data provider https://www.alphavantage.co/ 

Data provider https://polygon.io/stocks

Data provider https://data.nasdaq.com/

NASDAQ Futures https://data.nasdaq.com/data/EUREX-eurex-futures-data

ECB Data Warehouse https://sdw.ecb.europa.eu/reports.do?node=1000003300


Packages
--------------------
quandl (https://data.nasdaq.com/tools/python)

In [2]:
import numpy as np
import pandas as pd
import quandl
import datacollection as daco
import datetime
%load_ext autoreload
%autoreload 2

In [44]:
def data_dl (dataref="", date="latest"):
    """
    Universal data download function that handles the different API calls based
    on what is being requested. Will ask for your identifiers where required.
    dataref: to provide the reference of the data to extract. The reference can be found in the datacatalogue.
    date: 'latest' will use today as reference point and pull out the latest available data
    Returns a time series.
    Leave dataref blank for dictionary of available sources. 
    """
   
    # Overall logic
    # 1. Provide datareference and dates
    #   a. If you don't know them you can retrieve a full list of what is available in the function
    #   b. But for this list you need a consolidated list of catalogues!
    # 2. The function retrieves the codes to get the data so you do not have to remember
    #   a. Retrieves the codes from files for the service requested
    #   b. If not available on the file then goes and download them from the support location
    #   c. Builds the request
    # 3. Sends the request to the API
    # 4. If the time period isn't available then the next best?
    # OUTPUT: data file downloaded

    # To do - use the data catalogue function here
    available = [
        "NASDAQ EUREX Futures"
    ]
    
    # Loads up all the available API to only have one call to the file
    api_dictionary = daco.apidict("read")
    
    # Works out the time period requested
    # There will be adjustments made to the date to match the specific requirements of the API queries
    if date == "latest":
         date = datetime.datetime.now()
    
    # Shows data sources for which API calls are set up
    # Otherwise proceeds with the downloads
    if dataref == "":
        return available

    elif dataref == "NASDAQ EUREX Futures":
        import quandl
        
        # Sets up the API based on the dictionary
        quandl.ApiConfig.api_key = api_dictionary["NASDAQ"]
        
        # Futures delivery month codes
        futures_delivery_month = {
                "January": "F",
                "February": "G",
                "March": "H",
                "April": "J",
                "May": "K",
                "June": "M",
                "July": "N",
                "August": "Q",
                "September": "U",
                "October": "V",
                "November": "X",
                "December": "Z"
        }

        # Building the query
        # To do - retrieve the code dynically via the catalogue
        query = {
            "code" : "EUREX/FVS",
            "month" : "",
            "year" : ""
        }

        # Translating the data to the futures format
        query['year'] = date.strftime('%Y')
        query['month'] = futures_delivery_month[date.strftime('%B')]
        query = query["code"] + query["month"] + query["year"]

        # Limiting to the 10 previous days
        # Converting to strings read by the API
        date_delta = datetime.timedelta(days=-10)
        start_date = date + date_delta
        start_date = start_date.strftime('%Y') + "-" + start_date.strftime('%m') + "-" + start_date.strftime('%d')
        end_date = date.strftime('%Y') + "-" + date.strftime('%m') + "-" + date.strftime('%d')

        data = quandl.get('EUREX/FVSJ2022', column_index='1',start_date=start_date, end_date=end_date)
    
    return data



In [45]:
data = data_dl("NASDAQ EUREX Futures")
data

,Open
Date,
2021-11-24,25.25
2021-11-25,24.75
2021-11-26,26.00
2021-11-29,25.95
2021-11-30,27.20
2021-12-01,25.15
2021-12-02,26.60
2021-12-03,27.30


In [33]:
date = datetime.datetime.now()

# Futures delivery month codes
futures_delivery_month = {
        "January": "F",
        "February": "G",
        "March": "H",
        "April": "J",
        "May": "K",
        "June": "M",
        "July": "N",
        "August": "Q",
        "September": "U",
        "October": "V",
        "November": "X",
        "December": "Z"
}

# Building the query
# To do - retrieve the code dynically via the catalogue
query = {
    "code" : "EUREX/FVS",
    "month" : "",
    "year" : ""
}

# Translating the data to the futures format
query['year'] = date.strftime('%Y')
query['month'] = futures_delivery_month[date.strftime('%B')]
query = query["code"] + query["month"] + query["year"]

# Limiting to the 10 previous days
# Converting to strings read by the API
date_delta = datetime.timedelta(days=-10)
start_date = date + date_delta
start_date = start_date.strftime('%Y') + "-" + start_date.strftime('%m') + "-" + start_date.strftime('%d')
end_date = date.strftime('%Y') + "-" + date.strftime('%m') + "-" + date.strftime('%d')


'2021-11-24'

In [42]:
api_dictionary = daco.apidict("read")
api_dictionary["NASDAQ"]


'pEqstxse_jcm7fkyXfyh'

In [43]:
quandl.ApiConfig.api_key('pEqstxse_jcm7fkyXfyh')

TypeError: 'NoneType' object is not callable

In [1]:
def datacat_admin(rebuild):
    """
    To administer the single data catalogue, which contains all the codes covered by the data retrieval functions.
    rebuild: True/False, to recreate the data catalogue based on the contents of the nasdaq_eurex
    """
    
    import os
    
    # Overall logic
    # A. (re) build the data catalogue
    #   1. Lists the files in the docs folder
    #   2. Opens each file and looks for the standardisation identifiers
    #   3. Extracts the standardisation identifiers
    #   4. Save the identifiers into the catalogue dataframe
    #   5. Restart fo the next file
    #   6. Saves the data catalogue in the doc folder
    # B. No idea yet
    
    # 1. Lists the files in the docs folder
    path = os.getcwd()
    path = path.replace("\\src","\\docs")
    files = os.listdir(path)
    
    # Removing '.ipynb_checkpoints'
    files = list(filter(lambda k: not'.ipynb_checkpoints' in k,files))
    
    # 2. Opens each file and looks for the standardisation identifiers
    content = n
    
    
    
    
    

### Now working on the standardisation of the documentation files

In [24]:
def docu_standardise(docu, meta_provider, meta_desc, meta_product):
    """
    Adds columns and values to data services documentation files so it can be consolidated into the single data catalogue
    """
    
    # 1. Receives a dataframe of the documentation file and the filename. The latter will be used to add the identifiers/metadata
    # 2. Uses the metada data to add the identifying columns
    # 3. Send back the documementation with the identifying columns
    
    

In [6]:
daco.docu_dl(source="")

Documentation available on the following:
['NASDAQ EUREX Futures']


In [14]:
docu = daco.docu_dl('NASDAQ EUREX Futures',save=True)

In [17]:
df = docu
file_name = "nasdaq_eurex_futures.csv"
meta_provider = "nasdaq"
meta_desc = "eurex"
meta_product = "futures"

# Adding the new columns with the details
docu["meta_provider"] = meta_provider
docu["meta_desc"] = meta_desc
docu["meta_product"] = meta_product




,Symbol,Exchange,Name,Months,Quandl Code,meta_provider,meta_desc,meta_subdesc
0,CONF,EUREX,CONF,HMUZ,EUREX/CONF,nasdaq,eurex,futures
1,F2CR,EUREX,CER Emissions Mid December,Z,EUREX/F2CR,nasdaq,eurex,futures
2,F2MX,EUREX,MDAX,HMUZ,EUREX/F2MX,nasdaq,eurex,futures
3,FBTP,EUREX,Long-Term Euro-BTP,HMUZ,EUREX/FBTP,nasdaq,eurex,futures
4,FBTS,EUREX,Short-Term Euro-BTP,HMUZ,EUREX/FBTS,nasdaq,eurex,futures
...,...,...,...,...,...,...,...,...
66,FSMP,EUREX,Skimmed Milk Powder (EUREX),FGHJKMNQUVXZ,EUREX/FSMP,nasdaq,eurex,futures
67,FSTX,EUREX,STOXX Europe 50 Index,HMUZ,EUREX/FSTX,nasdaq,eurex,futures
68,FTDX,EUREX,DivDAX,HMUZ,EUREX/FTDX,nasdaq,eurex,futures
69,FVS,EUREX,EURO STOXX 50 Volatility (VSTOXX),HMUZ,EUREX/FVS,nasdaq,eurex,futures
